# A Custom Multi Layer Perceptron For Time Series Classification

### Importing Three Different Time Series Datasets by using sktime's `load_UCR_UEA_dataset` 
#### About the datasets:
* First dataset is called **DistalPhalanxOutlineCorrect**. It has 2 classes, It is of type Image, and it has one dimension.
* Second dataset is called **ItalyPowerDemand**. It has 2 classes, It is of type Sensor, and it has one dimension.
* Third dataset is called **BME**. It has 3 classes, It is of type Simulated, and it has one dimension.

In [ ]:
# ------ Importing necessary packages!
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# load_UCR_UEA_dataset
from sktime.datasets import load_UCR_UEA_dataset

In [2]:
# ------ Importing Time Series Datasets!
# 2 Classes, 1D, Image
X_train_1, y_train_1 = load_UCR_UEA_dataset(name="DistalPhalanxOutlineCorrect", split="train", return_type="numpy2D")
X_test_1, y_test_1 = load_UCR_UEA_dataset(name="DistalPhalanxOutlineCorrect", split="test", return_type="numpy2D")

# 2 Classes, 1D, Sensor 
X_train_2, y_train_2 = load_UCR_UEA_dataset(name= "ItalyPowerDemand", split="train", return_type="numpy2D")
X_test_2, y_test_2 = load_UCR_UEA_dataset(name="ItalyPowerDemand", split="test", return_type="numpy2D")

# 3Classes, 1D, Simulated
X_train_3, y_train_3 = load_UCR_UEA_dataset(name = "BME", split="train", return_type="numpy2D")
X_test_3, y_test_3 = load_UCR_UEA_dataset(name = "BME", split="test", return_type="numpy2D")

In [3]:
print("Dataset 1:", X_train_1.shape, X_test_1.shape)
print("Dataset 2:", X_train_2.shape, X_test_2.shape)
print("Dataset 3:", X_train_3.shape, X_test_3.shape)

Dataset 1: (600, 80) (276, 80)
Dataset 2: (67, 24) (1029, 24)
Dataset 3: (30, 128) (150, 128)


We have different datasets, which means we must have a function that can take in different datasets with different length.  

The plain baseline is a basic **Multi Layer Perceptron** which is build by stacking three fully connected layers. 

The fully connected layers **each has 500 neurons** following 2 designs roles:
* Using **Dropout** at each layer's input to improve the generalization capability!  
    * The fraction of the input units to drop on the input layer is **0.1**
    * The fraction of the input units to drop on the second and third layers is **0.2**
    * The fraction of the input units to drop on the output layer is **0.3**
* Using **ReLU** as activation function to prevent saturation of the gradient when the network is deep!  
And the output layer ends the network with a **softmax** activation function!

The MLP Model is then trained with **Adadelta** with learning rate 0.1, ρ = 0.95 and e = 1e − 8!  

The loss function for the model is **categorical cross entropy**! With **accuracy** as metrics!

In [4]:
def create_mlp_model(input_shape, num_classes):
    mlp_model = tf.keras.Sequential([
        keras.layers.Dropout(0.1, input_shape= input_shape),
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(num_classes, activation="softmax")
    ])
    mlp_model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.1, rho=0.95, epsilon=1e-8),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return mlp_model

Defining **callbacks** parameters for fine tuning!  
Defining **label encoder** for converting data to numerical ones.

In [7]:
# Converting to numerical variables.
le = LabelEncoder() 
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
]

Creating training sets by using **train_test_split()** to split the data into random train and test subsets!  
* test_size = 0.2! 
    * 20% of the data will be used for the valiation/test set and the remaining 80% will be used for the training set!
* random_state=42!
    * Ensuring that the splits that generates are reproducible!

In [8]:
# --------- Creating and training MLP on the FIRST dataset ----------- 
X_train_MLP_1, X_val_MLP_1, y_train_MLP_1, y_val_MLP_1 = train_test_split(X_train_1, y_train_1, test_size=0.2, random_state=42)
# --------- Creating and training MLP on the SECOND dataset ----------- 
X_train_MLP_2, X_val_MLP_2, y_train_MLP_2, y_val_MLP_2 = train_test_split(X_train_2, y_train_2, test_size=0.2, random_state=42)
# ------------- Creating and training MLP on the THIRD dataset ------------
X_train_MLP_3, X_val_MLP_3, y_train_MLP_3, y_val_MLP_3 = train_test_split(X_train_3, y_train_3, test_size=0.2, random_state=42)

##### Preparing the training sets for the models or training!

In [9]:
# --------- First Training set, TYPE=Image ------------
# One-hot encode the labels
y_train_MLP_1 = tf.keras.utils.to_categorical(y_train_MLP_1, num_classes=np.unique(y_train_MLP_1).shape[0])
y_val_MLP_1 = tf.keras.utils.to_categorical(y_val_MLP_1, num_classes=np.unique(y_train_MLP_1).shape[0])
# Reshaping the dataset (batch_size, steps)
X_train_MLP_1 = X_train_MLP_1.reshape(X_train_MLP_1.shape[0], -1)
X_val_MLP_1 = X_val_MLP_1.reshape(X_val_MLP_1.shape[0], -1)

# --------- Second Training set! TYPE=Sensor ------------
# Converting to int.
y_train_MLP_2 = le.fit_transform(y_train_MLP_2) #Converting to int
y_val_MLP_2 = le.transform(y_val_MLP_2) #Converting to int
# One-hot encode the labels
y_train_MLP_2 = tf.keras.utils.to_categorical(y_train_MLP_2, num_classes=int(np.max(y_train_MLP_2) + 1))
y_val_MLP_2 = tf.keras.utils.to_categorical(y_val_MLP_2, num_classes= int(np.max(y_train_MLP_2) + 1))
# Reshaping the dataset (batch_size, steps)
X_train_MLP_2 = X_train_MLP_2.reshape(X_train_MLP_2.shape[0], -1)
X_val_MLP_2 = X_val_MLP_2.reshape(X_val_MLP_2.shape[0], -1)

# ---------- Third Training set! TYPE=Simulated ------------
# Converting to int.
y_train_MLP_3 = le.fit_transform(y_train_MLP_3) #Converting to int
y_val_MLP_3 = le.transform(y_val_MLP_3) #Converting to int
# Subtract 1 from the labels to make them start from 0
y_train_MLP_3 = y_train_MLP_3 - 1
y_val_MLP_3 = y_val_MLP_3 - 1
# One-hot encode the labels
y_train_MLP_3 = tf.keras.utils.to_categorical(y_train_MLP_3, num_classes=int(np.max(y_train_MLP_3) + 1))
y_val_MLP_3 = tf.keras.utils.to_categorical(y_val_MLP_3, num_classes=int(np.max(y_train_MLP_3) + 1))
# Reshaping the dataset (batch_size, steps)
X_train_MLP_3 = X_train_MLP_3.reshape(X_train_MLP_3.shape[0], -1)
X_val_MLP_3 = X_val_MLP_3.reshape(X_val_MLP_3.shape[0], -1)

##### Slicing and shuffling the training sets!

In [13]:
# ------------ First Training Set! ---------------
# Convert the numpy arrays to a tf.data.Dataset
train_dataset_MLP_1 = tf.data.Dataset.from_tensor_slices((X_train_MLP_1, y_train_MLP_1))
val_dataset_MLP_1 = tf.data.Dataset.from_tensor_slices((X_val_MLP_1, y_val_MLP_1))
# Shuffle, batch, and prefetch the dataset
train_dataset_MLP_1 = train_dataset_MLP_1.shuffle(buffer_size=1024).batch(64).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset_MLP_1 = val_dataset_MLP_1.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

# ----------- Second Training Set! --------------
# Convert the numpy arrays to a tf.data.Dataset
train_dataset_MLP_2 = tf.data.Dataset.from_tensor_slices((X_train_MLP_2, y_train_MLP_2))
val_dataset_MLP_2 = tf.data.Dataset.from_tensor_slices((X_val_MLP_2, y_val_MLP_2))
# Shuffle, batch, and prefetch the dataset
train_dataset_MLP_2 = train_dataset_MLP_2.shuffle(buffer_size=1024).batch(64).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset_MLP_2 = val_dataset_MLP_2.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

# ------------ Third Training Set! ---------------
# Convert the numpy arrays to a tf.data.Dataset
train_dataset_MLP_3 = tf.data.Dataset.from_tensor_slices((X_train_MLP_3, y_train_MLP_3))
val_dataset_MLP_3 = tf.data.Dataset.from_tensor_slices((X_val_MLP_3, y_val_MLP_3))
# Shuffle, batch, and prefetch the dataset
train_dataset_MLP_3 = train_dataset_MLP_3.shuffle(buffer_size=1024).batch(64).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset_MLP_3 = val_dataset_MLP_3.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

#### Creating and Fitting the custom MLP model for the first dataset!
* Training the model on the first dataset!

In [ ]:
# Create the model
model_MLP_1 = create_mlp_model(X_train_MLP_1.shape[1:], np.unique(y_train_MLP_1).shape[0])
# Train the model
model_MLP_1.fit(train_dataset_MLP_1, epochs=10, validation_data=val_dataset_MLP_1, callbacks=callbacks)

##### Testing the model on unseen data!
* Pre-Processing The Test Data

In [34]:
# One-hot encode the test labels are not needed because they are already one-hot encoded!
# Applying to_categorical would add an extra dimension!
# Reshape the test dataset
X_test_11 = X_test_1.reshape(X_test_1.shape[0], -1)
# Convert the numpy arrays to a tf.data.Dataset
test_dataset_MLP_1 = tf.data.Dataset.from_tensor_slices((X_test_11, y_test_1))
# Batch and prefetch the dataset
test_dataset_MLP_1 = test_dataset_MLP_1.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

* Evaluating the Custom MLP model on the **first** dataset!
    * First dataset is called **DistalPhalanxOutlineCorrect**. It has 2 classes, It is of type Image, and it has one dimension.

In [35]:
# Evaluate the model
test_loss, test_accuracy = model_MLP_1.evaluate(test_dataset_MLP_1)
print(f"Test accuracy: {test_accuracy}")
print(f"Test loss: {test_loss}")

5/5 [==============================] - 0s 5ms/step - loss: 0.6766 - accuracy: 0.5870
Test accuracy: 0.5869565010070801
Test loss: 0.6766079664230347


#### Creating and Fitting the custom MLP model for the second dataset!
* Training the model on the second dataset!

In [ ]:
# Create the model
model_MLP_2 = create_mlp_model(X_train_MLP_2.shape[1:], np.unique(y_train_MLP_2).shape[0])
# Train the model
model_MLP_2.fit(train_dataset_MLP_2, epochs=10, validation_data=val_dataset_MLP_2, callbacks=callbacks)

##### Testing the model on unseen data!
* Pre-Processing The Test Data

In [43]:
# Convert the labels to integers
y_test_22 = le.transform(y_test_2)
# One-hot encode the test labels
y_test_22 = tf.keras.utils.to_categorical(y_test_22, num_classes=int(np.max(y_train_MLP_2) + 1))

# Reshape the test dataset
X_test_22 = X_test_2.reshape(X_test_2.shape[0], -1)
# Convert the numpy arrays to a tf.data.Dataset
test_dataset_MLP_2 = tf.data.Dataset.from_tensor_slices((X_test_22, y_test_22))
# Batch and prefetch the dataset
test_dataset_MLP_2 = test_dataset_MLP_2.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

* Evaluating the Custom MLP model on the **second** dataset!
    * Second dataset is called **ItalyPowerDemand**. It has 2 classes, It is of type Sensor, and it has one dimension.

In [44]:
# Evaluate the model
test_loss, test_accuracy = model_MLP_2.evaluate(test_dataset_MLP_2)
print(f"Test accuracy: {test_accuracy}")
print(f"Test loss: {test_loss}")

17/17 [==============================] - 0s 5ms/step - loss: 0.6544 - accuracy: 0.5228
Test accuracy: 0.5228376984596252
Test loss: 0.65443354845047


#### Creating and Fitting the custom MLP model for the third dataset!
* Training the model on the third dataset!

In [ ]:
# Create the model
model_MLP_3 = create_mlp_model(X_train_MLP_3.shape[1:], np.unique(y_train_MLP_3).shape[0])
# Train the model
model_MLP_3.fit(train_dataset_MLP_3, epochs=10, validation_data=val_dataset_MLP_3, callbacks=callbacks)

##### Testing the model on unseen data!
* Pre-Processing The Test Data

In [52]:
# Convert the labels to integers and subtract 1
y_test_33 = le.transform(y_test_3) - 1
# One-hot encode the test labels
y_test_33 = tf.keras.utils.to_categorical(y_test_33, num_classes=int(np.max(y_train_MLP_3) + 1))
#  Reshape the test dataset
X_test_33 = X_test_3.reshape(X_test_3.shape[0], -1)
# Convert the numpy arrays to a tf.data.Dataset
test_dataset_MLP_3 = tf.data.Dataset.from_tensor_slices((X_test_33, y_test_33))
# Batch and prefetch the dataset
test_dataset_MLP_3 = test_dataset_MLP_3.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

* Evaluating the Custom MLP model on the **third** dataset!
    * Third dataset is called **BME**. It has 3 classes, It is of type Simulated, and it has one dimension.

In [53]:
# Evaluate the model
test_loss, test_accuracy = model_MLP_3.evaluate(test_dataset_MLP_3)
print(f"Test accuracy: {test_accuracy}")
print(f"Test loss: {test_loss}")

3/3 [==============================] - 0s 7ms/step - loss: 0.6011 - accuracy: 0.5267
Test accuracy: 0.5266666412353516
Test loss: 0.6010997295379639
